In [3]:
import pandas as pd

import requests
from requests import get
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver
import time
import timeit

import numpy as np

import re

In [4]:
headers = {"Accept-Language": "en-US, en;q=0.5"}
direction = ["North","East","West","South","E","W","N","S"]

In [8]:
## Getting all the links from the website

options = Options()
options.add_argument('--headless')
options.add_argument('--disable-gpu')
driver=webdriver.Chrome(executable_path = 'C:/Users/kuria/anaconda3/Lib/site-packages/selenium/webdriver/chrome/chromedriver.exe',options = options)
driver.get("https://www.rentcollegepads.com/off-campus-housing/ohio-state/search")
listings=[]

while(True):
    try:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        stages = driver.find_elements_by_class_name('col-9')
        page = driver.page_source
        soup = BeautifulSoup(page, 'html.parser')
        container = soup.find_all('div', attrs={'class':'col-9'})
    
        len_list_before = len(listings)
  
        for div in container:
            if div.a["href"] in listings:
                continue
            else:
                listings.append(div.a['href'])

        len_list_after = len(listings)
    
        if len_list_before==len_list_after:
            break

        loadmore = driver.find_element_by_css_selector('button[class = "btn btn-success btn-block"]')
        loadmore.click()
        time.sleep(2)
        
    except NoSuchElementException: 
        print("Reached the end")
        end = timeit.timeit()
        break
        

Reached the end
0.0005301999999574036


In [5]:
## Writing the links to a csv
df_listing = pd.read_csv('Listing_1.csv')

In [181]:
name = []
location_name = []
location = []
stuff = []
names = []
bed = []
bath = []
rent = []
available = []
sq_ft = []

for i in range(0,len(df),1):
    url = df['urls'][i]
    page = requests.get(url ,headers = headers)
    soup = BeautifulSoup(page.text, "html.parser")    
    table = soup.find_all('div', 
              attrs = {'class':'tab-pane active',
                      'aria-hidden':"false",
                      'id':"__BVID__43"})

    for row in table[0].find_all('tr'):
        for col in row.find_all('td'):
            if not col.string is None:
                stuff.append(col.string.strip())
            else:
                stuff.append("None")
         
        if not stuff:
            continue
        else:
            names.append(stuff[0])
            bed.append(stuff[1])
            bath.append(stuff[2])
            rent.append(stuff[3])
            sq_ft.append(stuff[4])
            available.append(stuff[5])
        
        location_name.append(soup.find("span", re.compile("sub-heading")).text)
        name = soup.find("span", re.compile("sub-heading")).text
        if not list(set(name.split()) & set(direction)):
            location.append("Null")
        else:
            location.append(list(set(name.split()) & set(direction))[0])
    
        stuff = []


In [182]:
units = pd.DataFrame(columns = ['location_name','location','name','beds','baths','price','available','sq_ft'])

units['location_name'] = location_name
units['location'] = location
units['name'] = names
units['beds'] = bed
units['baths'] = bath
units['price'] = rent
units['available'] = available
units['sq_ft'] = sq_ft

In [183]:
units

,location_name,location,name,beds,baths,price,available,sq_ft
0,"\n442 East Northwood Avenue Columbus, OH 43201...",East,Unfurnished,2,1,790-790,2020-08-15,None
1,"\n442 East Northwood Avenue Columbus, OH 43201...",East,Furnished,2,1,850-850,2020-08-15,None
2,"\n250 W Norwich Ave Columbus, OH 43201\n",W,Duo,2,2,1130-1140,0000-00-00,960
3,"\n250 W Norwich Ave Columbus, OH 43201\n",W,Quartet 1,4,4,950-965,0000-00-00,1549
4,"\n250 W Norwich Ave Columbus, OH 43201\n",W,Quartet 2,4,4,920-955,0000-00-00,1481
...,...,...,...,...,...,...,...,...
1389,"\n155 W 10th Ave, Columbus, OH 43201, United S...",W,None,6,2,0,0000-00-00,None
1390,"\n393 W 8th Ave Columbus, OH 43201\n",W,393 W 8th Ave,7,3,4000-4000,2021-07-25,None
1391,"\n50 W Oakland Ave, Columbus, OH 43201, United...",W,4,1,1,599-599,2020-07-04,None
1392,"\n201 E 16th Ave Columbus, OH 43201 USA\n",E,5x2,5,2,2500-2500,2018-12-01,1352


In [217]:
stuff = []
parking_type = []
number = []
price = []
price_type = []
location_name = []
random_0 = []
random_1 = []
random_2 = []
random_3 = []

for i in range(0,len(df),1):
    url = df['urls'][i]
    page = requests.get(url ,headers = headers)
    soup = BeautifulSoup(page.text, "html.parser")
    feature_block = soup.find_all('div',
             attrs = {'class':'feature-block'})[1]
    
    check_description = feature_block.find_all('h2')
    if not check_description[0].string :
        for row in feature_block.find_all('tr'):
            for col in row.find_all('td'):
                if not col.string is None:
                    stuff.append(col.string.strip())
                else:
                    stuff.append("None")
         
            if not stuff:
                continue
            else:
                parking_type.append(stuff[0])
                number.append(stuff[1])
                price.append(stuff[2])
                price_type.append(stuff[3])

            stuff = []
            
            location_name.append(soup.find("span", re.compile("sub-heading")).text)
            
    else:

        for li in feature_block.find_all('li'):
            if 'Laundry' in li.text:
                random_1.append(li.text)
                random_1.append(url)
                random_1.append(soup.find("span", re.compile("sub-heading")).text)
            else:
                random_1.append('Null')
            if 'Furnished' in li.text:
                random_2.append(li.text)
                random_2.append(url)
                random_2.append(soup.find("span", re.compile("sub-heading")).text)
            else:
                random_2.append('Null')
            if 'Parking' in li.text:
                random_3.append(li.text)
                random_3.append(url)
                random_3.append(soup.find("span", re.compile("sub-heading")).text)
            else:
                random_3.append('Null')
        
       # random.append(soup.find("span", re.compile("sub-heading")).text)
    

In [219]:
parking = pd.DataFrame(columns = ['location_name','parking_type', 'number','price','price_type'])
parking['location_name'] = location_name
parking['parking_type'] = parking_type
parking['number'] = number
parking['price_parking'] = price
parking['price_type'] = price_type

In [222]:
units

,location_name,location,name,beds,baths,price,available,sq_ft
0,"\n442 East Northwood Avenue Columbus, OH 43201...",East,Unfurnished,2,1,790-790,2020-08-15,None
1,"\n442 East Northwood Avenue Columbus, OH 43201...",East,Furnished,2,1,850-850,2020-08-15,None
2,"\n250 W Norwich Ave Columbus, OH 43201\n",W,Duo,2,2,1130-1140,0000-00-00,960
3,"\n250 W Norwich Ave Columbus, OH 43201\n",W,Quartet 1,4,4,950-965,0000-00-00,1549
4,"\n250 W Norwich Ave Columbus, OH 43201\n",W,Quartet 2,4,4,920-955,0000-00-00,1481
...,...,...,...,...,...,...,...,...
1389,"\n155 W 10th Ave, Columbus, OH 43201, United S...",W,None,6,2,0,0000-00-00,None
1390,"\n393 W 8th Ave Columbus, OH 43201\n",W,393 W 8th Ave,7,3,4000-4000,2021-07-25,None
1391,"\n50 W Oakland Ave, Columbus, OH 43201, United...",W,4,1,1,599-599,2020-07-04,None
1392,"\n201 E 16th Ave Columbus, OH 43201 USA\n",E,5x2,5,2,2500-2500,2018-12-01,1352


In [223]:
parking

,location_name,parking_type,number,price,price_type
0,"\n442 East Northwood Avenue Columbus, OH 43201...",Off-Street Parking,1,$100,Included
1,"\n250 W Norwich Ave Columbus, OH 43201\n",Garage Parking,1,$100,Monthly
2,"\n1234 Steelwood Rd Columbus, OH 43212\n",Surface Lot,1,Call For Price,FREE
3,"\n1234 Steelwood Rd Columbus, OH 43212\n",Garage Parking,1,$50,Monthly
4,"\n1234 Steelwood Rd Columbus, OH 43212\n",Indoor Bike Storage,2,Call For Price,FREE
...,...,...,...,...,...
499,"\n1656 Summit St, Columbus, OH 43201, United S...",Off-Street Parking,0,Call For Price,Included
500,"\n43 East 15th Avenue Columbus, OH 43201 USA\n",Off-Street Parking,0,Call For Price,Included
501,"\n50 W Oakland Ave, Columbus, OH 43201, United...",Off-Street Parking,0,Call For Price,Included
502,"\n201 E 16th Ave Columbus, OH 43201 USA\n",Off-Street Parking,2,$300,Annually


In [319]:
parking_new = pd.DataFrame(columns = ['location_name','parking_type','number'])
loc_name = []
park = []
i = 0
while  i < len(random_3): 
    if random_3[i] != 'Null':
        loc_name.append(random_3[i+2])
        park.append(random_3[i])
        i+=3
    else:
        i+=1

parking_new['location_name'] = loc_name
parking_new['parking_type'] = park
parking_new.loc[:,'number'] = 1
parking_new

,location_name,parking_type,number
0,"\n151 W 10th Ave, Columbus, OH 43201, United S...",On Street Parking Only,1
1,"\n116 E 12th Ave, Columbus, OH 43201, United S...",Off Street Parking,1
2,"\n253 E 19th Ave, Columbus, OH 43201, United S...",Assigned Parking Space,1
3,"\n23 W 10th Ave\nColumbus, OH 43201\n",Off Street Parking,1
4,"\n95 Chittenden Ave, Columbus, OH 43201, Unite...",2 Assigned Parking Spaces (1 Double Lane),1
5,"\n94 E 18th Ave, Columbus, OH 43201, United St...",2 Assigned Parking Spaces (1 Double Lane),1
6,"\n214 W Norwich Ave, Columbus, OH 43201, Unite...",Assigned Off-Street Parking Spaces,1
7,"\n155 W 10th Ave, Columbus, OH 43201, United S...",On Street Parking Only,1


In [321]:
parking = pd.concat([parking, parking_new], axis=0)

In [333]:
parking

,location_name,parking_type,number,parking_price,price_type
0,"\n442 East Northwood Avenue Columbus, OH 43201...",Off-Street Parking,1,$100,Included
1,"\n250 W Norwich Ave Columbus, OH 43201\n",Garage Parking,1,$100,Monthly
2,"\n1234 Steelwood Rd Columbus, OH 43212\n",Surface Lot,1,Call For Price,FREE
3,"\n1234 Steelwood Rd Columbus, OH 43212\n",Garage Parking,1,$50,Monthly
4,"\n1234 Steelwood Rd Columbus, OH 43212\n",Indoor Bike Storage,2,Call For Price,FREE
...,...,...,...,...,...
3,"\n23 W 10th Ave\nColumbus, OH 43201\n",Off Street Parking,1,NaN,NaN
4,"\n95 Chittenden Ave, Columbus, OH 43201, Unite...",2 Assigned Parking Spaces (1 Double Lane),1,NaN,NaN
5,"\n94 E 18th Ave, Columbus, OH 43201, United St...",2 Assigned Parking Spaces (1 Double Lane),1,NaN,NaN
6,"\n214 W Norwich Ave, Columbus, OH 43201, Unite...",Assigned Off-Street Parking Spaces,1,NaN,NaN


In [328]:
units

,location_name,location,name,beds,baths,price,available,sq_ft
0,"\n442 East Northwood Avenue Columbus, OH 43201...",East,Unfurnished,2,1,790-790,2020-08-15,None
1,"\n442 East Northwood Avenue Columbus, OH 43201...",East,Furnished,2,1,850-850,2020-08-15,None
2,"\n250 W Norwich Ave Columbus, OH 43201\n",W,Duo,2,2,1130-1140,0000-00-00,960
3,"\n250 W Norwich Ave Columbus, OH 43201\n",W,Quartet 1,4,4,950-965,0000-00-00,1549
4,"\n250 W Norwich Ave Columbus, OH 43201\n",W,Quartet 2,4,4,920-955,0000-00-00,1481
...,...,...,...,...,...,...,...,...
1389,"\n155 W 10th Ave, Columbus, OH 43201, United S...",W,None,6,2,0,0000-00-00,None
1390,"\n393 W 8th Ave Columbus, OH 43201\n",W,393 W 8th Ave,7,3,4000-4000,2021-07-25,None
1391,"\n50 W Oakland Ave, Columbus, OH 43201, United...",W,4,1,1,599-599,2020-07-04,None
1392,"\n201 E 16th Ave Columbus, OH 43201 USA\n",E,5x2,5,2,2500-2500,2018-12-01,1352


In [334]:
final_list = pd.merge(units, parking, on='location_name', how='outer')

In [335]:
pd.merge(units, parking, on='location_name', how='outer')

,location_name,location,name,beds,baths,price,available,sq_ft,parking_type,number,parking_price,price_type
0,"\n442 East Northwood Avenue Columbus, OH 43201...",East,Unfurnished,2,1,790-790,2020-08-15,None,Off-Street Parking,1,$100,Included
1,"\n442 East Northwood Avenue Columbus, OH 43201...",East,Furnished,2,1,850-850,2020-08-15,None,Off-Street Parking,1,$100,Included
2,"\n250 W Norwich Ave Columbus, OH 43201\n",W,Duo,2,2,1130-1140,0000-00-00,960,Garage Parking,1,$100,Monthly
3,"\n250 W Norwich Ave Columbus, OH 43201\n",W,Quartet 1,4,4,950-965,0000-00-00,1549,Garage Parking,1,$100,Monthly
4,"\n250 W Norwich Ave Columbus, OH 43201\n",W,Quartet 2,4,4,920-955,0000-00-00,1481,Garage Parking,1,$100,Monthly
...,...,...,...,...,...,...,...,...,...,...,...,...
1477,"\n155 W 10th Ave, Columbus, OH 43201, United S...",W,None,6,2,0,0000-00-00,None,On Street Parking Only,1,NaN,NaN
1478,"\n393 W 8th Ave Columbus, OH 43201\n",W,393 W 8th Ave,7,3,4000-4000,2021-07-25,None,NaN,NaN,NaN,NaN
1479,"\n50 W Oakland Ave, Columbus, OH 43201, United...",W,4,1,1,599-599,2020-07-04,None,Off-Street Parking,0,Call For Price,Included
1480,"\n201 E 16th Ave Columbus, OH 43201 USA\n",E,5x2,5,2,2500-2500,2018-12-01,1352,Off-Street Parking,2,$300,Annually


In [365]:
laundry = pd.DataFrame(columns = ['location_name','laundry_aval'])
loc_name = []
laundry_aval = []
i = 0
while  i < len(random_1): 
    if random_1[i] != 'Null':
        loc_name.append(random_1[i+2])
        laundry_aval.append(random_1[i])
        i+=3
    else:
        i+=1
laundry['location_name'] = loc_name
laundry['laundry_aval'] = laundry_aval
laundry

,location_name,laundry_aval
0,"\n51 E 13th Ave, Columbus, OH 43201, USA\n",Laundry in building
1,"\n151 W 10th Ave, Columbus, OH 43201, United S...",Laundry in Apartment
2,"\n116 E 12th Ave, Columbus, OH 43201, United S...",Laundry in House
3,"\n141 E 13th Ave, Columbus, OH 43201, USA\n",Laundry in Building
4,"\n253 E 19th Ave, Columbus, OH 43201, United S...",Laundry in Unit
5,"\n95 Chittenden Ave, Columbus, OH 43201, Unite...",Laundry in Unit
6,"\n94 E 18th Ave, Columbus, OH 43201, United St...",Laundry in Apartment
7,"\n99 E 16th Ave, Columbus, OH 43201, USA\n",Laundry on site
8,"\n214 W Norwich Ave, Columbus, OH 43201, Unite...",Laundry in Building
9,"\n155 W 10th Ave, Columbus, OH 43201, United S...",Laundry in Apartment


In [369]:
final_list = pd.merge(final_list,laundry,on = "location_name",how = "outer")
final_list.to_pickle("Please_work") 

In [1]:
import pickle
import pandas as pd
df = pd.read_pickle("Please_work")

In [6]:
df_listing

,Unnamed: 0,urls
0,0,https://www.rentcollegepads.com/city/columbus-...
1,1,https://www.rentcollegepads.com/city/columbus-...
2,2,https://www.rentcollegepads.com/city/columbus-...
3,3,https://www.rentcollegepads.com/city/columbus-...
4,4,https://www.rentcollegepads.com/city/columbus-...
...,...,...
593,593,https://www.rentcollegepads.com/city/columbus-...
594,594,https://www.rentcollegepads.com/city/columbus-...
595,595,https://www.rentcollegepads.com/city/columbus-...
596,596,https://www.rentcollegepads.com/city/columbus-...


In [137]:
additional = pd.DataFrame()
for i in range(0,len(df_listing)-1,1):
    Laundry = []
    Furnished = []
    Parking =  []
    Washer = []
    c = []
    thisdict = {}
    url = df_listing['urls'][i]
    page = requests.get(url ,headers = headers)
    soup = BeautifulSoup(page.text, "html.parser")
    feature_block = soup.find_all('div',
                 attrs = {'class':'extra-feature'})[2]
    
    for li in feature_block.find_all('li'):
             c.append(li.text.strip('\n'))
    if len(c) == 0:
        continue
    else:
        for j in c:
            if 'Laundry' in j:
                Laundry.append(j)
            if 'Furnished' in j:
                Furnished.append(j)
            if 'Parking' in j:
                Parking.append(j)
            if 'Washer' in j:
                Washer.append(j)
                    
    thisdict = {
      "url": url,
      "location_name": soup.find("span", re.compile("sub-heading")).text.strip('\n'),
      "Laundry": list(Laundry),
      "Furnished": list(Furnished),
      "Parking": list(Parking),
      "Washing": list(Washer)
    }
    
    additional = additional.append(thisdict, ignore_index=True)
                    

In [152]:
df

,location_name,location,name,beds,baths,price,available,sq_ft,parking_type,number,parking_price,price_type,laundry_aval
0,"442 East Northwood Avenue Columbus, OH 43201 USA",East,Unfurnished,2,1,790-790,2020-08-15,None,Off-Street Parking,1,$100,Included,NaN
1,"442 East Northwood Avenue Columbus, OH 43201 USA",East,Furnished,2,1,850-850,2020-08-15,None,Off-Street Parking,1,$100,Included,NaN
2,"250 W Norwich Ave Columbus, OH 43201",W,Duo,2,2,1130-1140,0000-00-00,960,Garage Parking,1,$100,Monthly,NaN
3,"250 W Norwich Ave Columbus, OH 43201",W,Quartet 1,4,4,950-965,0000-00-00,1549,Garage Parking,1,$100,Monthly,NaN
4,"250 W Norwich Ave Columbus, OH 43201",W,Quartet 2,4,4,920-955,0000-00-00,1481,Garage Parking,1,$100,Monthly,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1477,"155 W 10th Ave, Columbus, OH 43201, United States",W,None,6,2,0,0000-00-00,None,On Street Parking Only,1,NaN,NaN,Laundry in Apartment
1478,"393 W 8th Ave Columbus, OH 43201",W,393 W 8th Ave,7,3,4000-4000,2021-07-25,None,NaN,NaN,NaN,NaN,NaN
1479,"50 W Oakland Ave, Columbus, OH 43201, United S...",W,4,1,1,599-599,2020-07-04,None,Off-Street Parking,0,Call For Price,Included,NaN
1480,"201 E 16th Ave Columbus, OH 43201 USA",E,5x2,5,2,2500-2500,2018-12-01,1352,Off-Street Parking,2,$300,Annually,NaN


In [159]:
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
additional=additional.rename(columns = {'address':'location_name'})
additional['Furnished'] =  additional["Furnished"].apply(lambda x: ",".join(x) if isinstance(x, list) else x)
additional['Laundry'] =  additional["Laundry"].apply(lambda x: ",".join(x) if isinstance(x, list) else x)
additional['Washing'] =  additional["Washing"].apply(lambda x: ",".join(x) if isinstance(x, list) else x)
additional['Parking'] = additional["Parking"].apply(lambda x: "Null" if len(x) == 0 else x)

In [177]:
df = df.rename(columns = {'laundry_aval':'Laundry'})

In [182]:
additional

,Furnished,Laundry,Parking,Washing,location_name,url
0,"Furnished,Furnished Packages Available","Laundry Room,On-Site Laundry",Null,,"442 East Northwood Avenue Columbus, OH 43201 USA",https://www.rentcollegepads.com/city/columbus-...
1,"Furnished,Furnished Packages Available",On-Site Laundry,Null,WasherDryer In-Unit,"250 W Norwich Ave Columbus, OH 43201",https://www.rentcollegepads.com/city/columbus-...
2,Furnished,,Null,WasherDryer In-Unit,"1234 Steelwood Rd Columbus, OH 43212",https://www.rentcollegepads.com/city/columbus-...
3,,,Null,,"15 E Lane Ave, Columbus, OH 43201, United States",https://www.rentcollegepads.com/city/columbus-...
4,,On-Site Laundry,Null,,"4765 Blairfield Dr Columbus, OH 43214 USA",https://www.rentcollegepads.com/city/columbus-...
...,...,...,...,...,...,...
576,,,Null,,"43 East 15th Avenue Columbus, OH 43201 USA",https://www.rentcollegepads.com/city/columbus-...
577,,On-Site Laundry,Null,,"155 W 10th Ave, Columbus, OH 43201, United States",https://www.rentcollegepads.com/city/columbus-...
578,,,Null,,"393 W 8th Ave Columbus, OH 43201",https://www.rentcollegepads.com/city/columbus-...
579,,On-Site Laundry,Null,WasherDryer In-Unit,"50 W Oakland Ave, Columbus, OH 43201, United S...",https://www.rentcollegepads.com/city/columbus-...


In [174]:
df.drop(df.columns[[2, 6, 7,10,11]], axis=1)

,location_name,location,beds,baths,price,parking_type,number,laundry_aval
0,"442 East Northwood Avenue Columbus, OH 43201 USA",East,2,1,790-790,Off-Street Parking,1,NaN
1,"442 East Northwood Avenue Columbus, OH 43201 USA",East,2,1,850-850,Off-Street Parking,1,NaN
2,"250 W Norwich Ave Columbus, OH 43201",W,2,2,1130-1140,Garage Parking,1,NaN
3,"250 W Norwich Ave Columbus, OH 43201",W,4,4,950-965,Garage Parking,1,NaN
4,"250 W Norwich Ave Columbus, OH 43201",W,4,4,920-955,Garage Parking,1,NaN
...,...,...,...,...,...,...,...,...
1477,"155 W 10th Ave, Columbus, OH 43201, United States",W,6,2,0,On Street Parking Only,1,Laundry in Apartment
1478,"393 W 8th Ave Columbus, OH 43201",W,7,3,4000-4000,NaN,NaN,NaN
1479,"50 W Oakland Ave, Columbus, OH 43201, United S...",W,1,1,599-599,Off-Street Parking,0,NaN
1480,"201 E 16th Ave Columbus, OH 43201 USA",E,5,2,2500-2500,Off-Street Parking,2,NaN


In [215]:
merged = pd.merge(df,additional,on = "location_name",how = "left")
merged

,location_name,location,name,beds,baths,price,available,sq_ft,parking_type,number,parking_price,price_type,Laundry_x,Furnished,Laundry_y,Parking,Washing,url
0,"442 East Northwood Avenue Columbus, OH 43201 USA",East,Unfurnished,2,1,790-790,2020-08-15,None,Off-Street Parking,1,$100,Included,NaN,"Furnished,Furnished Packages Available","Laundry Room,On-Site Laundry",Null,,https://www.rentcollegepads.com/city/columbus-...
1,"442 East Northwood Avenue Columbus, OH 43201 USA",East,Furnished,2,1,850-850,2020-08-15,None,Off-Street Parking,1,$100,Included,NaN,"Furnished,Furnished Packages Available","Laundry Room,On-Site Laundry",Null,,https://www.rentcollegepads.com/city/columbus-...
2,"250 W Norwich Ave Columbus, OH 43201",W,Duo,2,2,1130-1140,0000-00-00,960,Garage Parking,1,$100,Monthly,NaN,"Furnished,Furnished Packages Available",On-Site Laundry,Null,WasherDryer In-Unit,https://www.rentcollegepads.com/city/columbus-...
3,"250 W Norwich Ave Columbus, OH 43201",W,Quartet 1,4,4,950-965,0000-00-00,1549,Garage Parking,1,$100,Monthly,NaN,"Furnished,Furnished Packages Available",On-Site Laundry,Null,WasherDryer In-Unit,https://www.rentcollegepads.com/city/columbus-...
4,"250 W Norwich Ave Columbus, OH 43201",W,Quartet 2,4,4,920-955,0000-00-00,1481,Garage Parking,1,$100,Monthly,NaN,"Furnished,Furnished Packages Available",On-Site Laundry,Null,WasherDryer In-Unit,https://www.rentcollegepads.com/city/columbus-...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1521,"155 W 10th Ave, Columbus, OH 43201, United States",W,None,6,2,0,0000-00-00,None,On Street Parking Only,1,NaN,NaN,Laundry in Apartment,,On-Site Laundry,Null,,https://www.rentcollegepads.com/city/columbus-...
1522,"393 W 8th Ave Columbus, OH 43201",W,393 W 8th Ave,7,3,4000-4000,2021-07-25,None,NaN,NaN,NaN,NaN,NaN,,,Null,,https://www.rentcollegepads.com/city/columbus-...
1523,"50 W Oakland Ave, Columbus, OH 43201, United S...",W,4,1,1,599-599,2020-07-04,None,Off-Street Parking,0,Call For Price,Included,NaN,,On-Site Laundry,Null,WasherDryer In-Unit,https://www.rentcollegepads.com/city/columbus-...
1524,"201 E 16th Ave Columbus, OH 43201 USA",E,5x2,5,2,2500-2500,2018-12-01,1352,Off-Street Parking,2,$300,Annually,NaN,,,Null,"WasherDryer In-Unit\n ,Washer Dryer Hook-Ups",https://www.rentcollegepads.com/city/columbus-...


In [219]:
merged['parking_type'].unique()

array(['Off-Street Parking', 'Garage Parking', 'Surface Lot',
       'Indoor Bike Storage', nan, 'Call for Availability',
       '404 King Parking', 'On-Site Parking', 'off-street parking',
       'Call For Availability', 'Off Street Parking',
       'Free Off Street Parking', 'Street Parking',
       'Free off street parking', 'On Street Parking Only',
       'Assigned Parking Space', 'Off Street Parking Available',
       'off-street parking available', 'Outdoor Bike Storage',
       'Gated Parking', '2 Assigned Parking Spaces (1 Double Lane)',
       'Garage & Surface Available', 'Driveway Parking',
       'Assigned Off-Street Parking Spaces', 'Underground Parking'],
      dtype=object)

In [235]:
merged['Parking_final'] = merged['parking_type'].where(merged['parking_type'].notnull(),merged['Parking'])
merged = merged.drop(['parking_type','Parking'],axis = 1)

In [230]:
merged[merged['parking_type'].isnull()]

,location_name,location,name,beds,baths,price,available,sq_ft,parking_type,number,parking_price,price_type,Laundry_x,Furnished,Laundry_y,Parking,Washing,url
24,"15 E Lane Ave, Columbus, OH 43201, United States",E,2x2,2,2,0,0000-00-00,None,NaN,NaN,NaN,NaN,NaN,,,Null,,https://www.rentcollegepads.com/city/columbus-...
25,"15 E Lane Ave, Columbus, OH 43201, United States",E,2x2 Den,2,2,0,0000-00-00,None,NaN,NaN,NaN,NaN,NaN,,,Null,,https://www.rentcollegepads.com/city/columbus-...
26,"15 E Lane Ave, Columbus, OH 43201, United States",E,3x3,3,3,0,0000-00-00,None,NaN,NaN,NaN,NaN,NaN,,,Null,,https://www.rentcollegepads.com/city/columbus-...
51,"62 E 13th Ave Columbus, OH 43201",E,C,4,1,2060-2060,2021-08-19,None,NaN,NaN,NaN,NaN,NaN,,,Null,,https://www.rentcollegepads.com/city/columbus-...
52,"62 E 13th Ave Columbus, OH 43201",E,D,4,1,2060-2060,2021-08-19,None,NaN,NaN,NaN,NaN,NaN,,,Null,,https://www.rentcollegepads.com/city/columbus-...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1497,"1382 Indianola Ave, Columbus, OH 43201, United...",Null,3x2,3,2,0,0000-00-00,None,NaN,NaN,NaN,NaN,NaN,,,Null,,https://www.rentcollegepads.com/city/columbus-...
1498,"1382 Indianola Ave, Columbus, OH 43201, United...",Null,4x2,4,2,0,0000-00-00,None,NaN,NaN,NaN,NaN,NaN,,,Null,,https://www.rentcollegepads.com/city/columbus-...
1515,"2159 Indianola Avenue Columbus, OH 43201",Null,2159 Indianola,3,2,1350-1450,2020-08-01,None,NaN,NaN,NaN,NaN,NaN,,,Null,WasherDryer In-Unit,https://www.rentcollegepads.com/city/columbus-...
1516,"2159 Indianola Avenue Columbus, OH 43201",Null,2157 Indianola,3,1,1300-1450,2020-08-01,1250,NaN,NaN,NaN,NaN,NaN,,,Null,WasherDryer In-Unit,https://www.rentcollegepads.com/city/columbus-...


In [241]:
merged['Laundry'] = merged['Laundry_x'].where(merged['Laundry_x'].notnull(),merged['Laundry_y'])
merged = merged.drop(['Laundry_x','Laundry_y'],axis = 1)

In [253]:
merged['Furnished'] =  additional["Furnished"].apply(lambda x: ",".join(x) if isinstance(x, list) else x)

,location_name,location,name,beds,baths,price,available,sq_ft,number,parking_price,price_type,Furnished,Washing,url,Parking_final,Laundry
12,"1234 Steelwood Rd Columbus, OH 43212",Null,1 Bedroom,1,1,1295-1395,2020-08-16,710-828,1,Call For Price,FREE,Furnished,WasherDryer In-Unit,https://www.rentcollegepads.com/city/columbus-...,Surface Lot,
13,"1234 Steelwood Rd Columbus, OH 43212",Null,1 Bedroom,1,1,1295-1395,2020-08-16,710-828,1,$50,Monthly,Furnished,WasherDryer In-Unit,https://www.rentcollegepads.com/city/columbus-...,Garage Parking,
14,"1234 Steelwood Rd Columbus, OH 43212",Null,1 Bedroom,1,1,1295-1395,2020-08-16,710-828,2,Call For Price,FREE,Furnished,WasherDryer In-Unit,https://www.rentcollegepads.com/city/columbus-...,Indoor Bike Storage,
15,"1234 Steelwood Rd Columbus, OH 43212",Null,2 Bedroom,2,2,900-900,2020-08-16,1065,1,Call For Price,FREE,Furnished,WasherDryer In-Unit,https://www.rentcollegepads.com/city/columbus-...,Surface Lot,
16,"1234 Steelwood Rd Columbus, OH 43212",Null,2 Bedroom,2,2,900-900,2020-08-16,1065,1,$50,Monthly,Furnished,WasherDryer In-Unit,https://www.rentcollegepads.com/city/columbus-...,Garage Parking,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1518,"1656 Summit St, Columbus, OH 43201, United States",Null,7x2,7,2,0,0000-00-00,None,0,Call For Price,Included,,,https://www.rentcollegepads.com/city/columbus-...,Off-Street Parking,
1519,"43 East 15th Avenue Columbus, OH 43201 USA",East,15x3.5,15,3.5,0,0000-00-00,None,0,Call For Price,Included,,,https://www.rentcollegepads.com/city/columbus-...,Off-Street Parking,
1520,"43 East 15th Avenue Columbus, OH 43201 USA",East,16x3.5,16,3.5,0,0000-00-00,None,0,Call For Price,Included,,,https://www.rentcollegepads.com/city/columbus-...,Off-Street Parking,
1522,"393 W 8th Ave Columbus, OH 43201",W,393 W 8th Ave,7,3,4000-4000,2021-07-25,None,NaN,NaN,NaN,,,https://www.rentcollegepads.com/city/columbus-...,Null,


In [258]:
merged[merged['Washing'].notnull()]

,location_name,location,name,beds,baths,price,available,sq_ft,number,parking_price,price_type,Furnished,Washing,url,Parking_final,Laundry,Laundry_Wash
0,"442 East Northwood Avenue Columbus, OH 43201 USA",East,Unfurnished,2,1,790-790,2020-08-15,None,1,$100,Included,"Furnished,Furnished Packages Available",,https://www.rentcollegepads.com/city/columbus-...,Off-Street Parking,"Laundry Room,On-Site Laundry","Laundry Room,On-Site Laundry"
1,"442 East Northwood Avenue Columbus, OH 43201 USA",East,Furnished,2,1,850-850,2020-08-15,None,1,$100,Included,"Furnished,Furnished Packages Available",,https://www.rentcollegepads.com/city/columbus-...,Off-Street Parking,"Laundry Room,On-Site Laundry","Laundry Room,On-Site Laundry"
2,"250 W Norwich Ave Columbus, OH 43201",W,Duo,2,2,1130-1140,0000-00-00,960,1,$100,Monthly,"Furnished,Furnished Packages Available",WasherDryer In-Unit,https://www.rentcollegepads.com/city/columbus-...,Garage Parking,On-Site Laundry,On-Site Laundry
3,"250 W Norwich Ave Columbus, OH 43201",W,Quartet 1,4,4,950-965,0000-00-00,1549,1,$100,Monthly,"Furnished,Furnished Packages Available",WasherDryer In-Unit,https://www.rentcollegepads.com/city/columbus-...,Garage Parking,On-Site Laundry,On-Site Laundry
4,"250 W Norwich Ave Columbus, OH 43201",W,Quartet 2,4,4,920-955,0000-00-00,1481,1,$100,Monthly,"Furnished,Furnished Packages Available",WasherDryer In-Unit,https://www.rentcollegepads.com/city/columbus-...,Garage Parking,On-Site Laundry,On-Site Laundry
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1520,"43 East 15th Avenue Columbus, OH 43201 USA",East,16x3.5,16,3.5,0,0000-00-00,None,0,Call For Price,Included,,,https://www.rentcollegepads.com/city/columbus-...,Off-Street Parking,,
1521,"155 W 10th Ave, Columbus, OH 43201, United States",W,None,6,2,0,0000-00-00,None,1,NaN,NaN,,,https://www.rentcollegepads.com/city/columbus-...,On Street Parking Only,Laundry in Apartment,Laundry in Apartment
1522,"393 W 8th Ave Columbus, OH 43201",W,393 W 8th Ave,7,3,4000-4000,2021-07-25,None,NaN,NaN,NaN,,,https://www.rentcollegepads.com/city/columbus-...,Null,,
1523,"50 W Oakland Ave, Columbus, OH 43201, United S...",W,4,1,1,599-599,2020-07-04,None,0,Call For Price,Included,,WasherDryer In-Unit,https://www.rentcollegepads.com/city/columbus-...,Off-Street Parking,On-Site Laundry,On-Site Laundry


In [264]:
merged['Washing'] = merged["Washing"].apply(lambda x: np.nan if x == '' else x)
merged['Laundry'] = merged["Laundry"].apply(lambda x: np.nan if x == '' else x)

In [273]:
merged['Laundry_Wash'] = merged['Washing'].where(merged['Washing'].notnull(),merged['Laundry'])
merged = merged.drop(['Laundry','Washing'],axis = 1)

In [276]:
merged['Furnished'] = merged["Furnished"].apply(lambda x: np.nan if x == '' else x)

In [279]:
merged = merged.drop(['available','sq_ft','parking_price','price_type'],axis = 1)
merged

,location_name,location,name,beds,baths,price,number,Furnished,url,Parking_final,Laundry_Wash
0,"442 East Northwood Avenue Columbus, OH 43201 USA",East,Unfurnished,2,1,790-790,1,"Furnished,Furnished Packages Available",https://www.rentcollegepads.com/city/columbus-...,Off-Street Parking,"Laundry Room,On-Site Laundry"
1,"442 East Northwood Avenue Columbus, OH 43201 USA",East,Furnished,2,1,850-850,1,"Furnished,Furnished Packages Available",https://www.rentcollegepads.com/city/columbus-...,Off-Street Parking,"Laundry Room,On-Site Laundry"
2,"250 W Norwich Ave Columbus, OH 43201",W,Duo,2,2,1130-1140,1,"Furnished,Furnished Packages Available",https://www.rentcollegepads.com/city/columbus-...,Garage Parking,WasherDryer In-Unit
3,"250 W Norwich Ave Columbus, OH 43201",W,Quartet 1,4,4,950-965,1,"Furnished,Furnished Packages Available",https://www.rentcollegepads.com/city/columbus-...,Garage Parking,WasherDryer In-Unit
4,"250 W Norwich Ave Columbus, OH 43201",W,Quartet 2,4,4,920-955,1,"Furnished,Furnished Packages Available",https://www.rentcollegepads.com/city/columbus-...,Garage Parking,WasherDryer In-Unit
...,...,...,...,...,...,...,...,...,...,...,...
1521,"155 W 10th Ave, Columbus, OH 43201, United States",W,None,6,2,0,1,NaN,https://www.rentcollegepads.com/city/columbus-...,On Street Parking Only,Laundry in Apartment
1522,"393 W 8th Ave Columbus, OH 43201",W,393 W 8th Ave,7,3,4000-4000,NaN,NaN,https://www.rentcollegepads.com/city/columbus-...,Null,NaN
1523,"50 W Oakland Ave, Columbus, OH 43201, United S...",W,4,1,1,599-599,0,NaN,https://www.rentcollegepads.com/city/columbus-...,Off-Street Parking,WasherDryer In-Unit
1524,"201 E 16th Ave Columbus, OH 43201 USA",E,5x2,5,2,2500-2500,2,NaN,https://www.rentcollegepads.com/city/columbus-...,Off-Street Parking,"WasherDryer In-Unit\n ,Washer Dryer Hook-Ups"


In [337]:
merged['Furnished'] = merged["Furnished"].apply(lambda x: np.nan if x == 'No' else x)
merged

,location_name,location,name,beds,baths,price,number,Furnished,url,Parking_final,Laundry_Wash
0,"442 East Northwood Avenue Columbus, OH 43201 USA",East,Unfurnished,2,1,790-790,1,"Furnished,Furnished Packages Available",https://www.rentcollegepads.com/city/columbus-...,Off-Street Parking,"Laundry Room,On-Site Laundry"
1,"442 East Northwood Avenue Columbus, OH 43201 USA",East,Furnished,2,1,850-850,1,"Furnished,Furnished Packages Available",https://www.rentcollegepads.com/city/columbus-...,Off-Street Parking,"Laundry Room,On-Site Laundry"
2,"250 W Norwich Ave Columbus, OH 43201",W,Duo,2,2,1130-1140,1,"Furnished,Furnished Packages Available",https://www.rentcollegepads.com/city/columbus-...,Garage Parking,WasherDryer In-Unit
3,"250 W Norwich Ave Columbus, OH 43201",W,Quartet 1,4,4,950-965,1,"Furnished,Furnished Packages Available",https://www.rentcollegepads.com/city/columbus-...,Garage Parking,WasherDryer In-Unit
4,"250 W Norwich Ave Columbus, OH 43201",W,Quartet 2,4,4,920-955,1,"Furnished,Furnished Packages Available",https://www.rentcollegepads.com/city/columbus-...,Garage Parking,WasherDryer In-Unit
...,...,...,...,...,...,...,...,...,...,...,...
1521,"155 W 10th Ave, Columbus, OH 43201, United States",W,None,6,2,0,1,NaN,https://www.rentcollegepads.com/city/columbus-...,On Street Parking Only,Laundry in Apartment
1522,"393 W 8th Ave Columbus, OH 43201",W,393 W 8th Ave,7,3,4000-4000,NaN,NaN,https://www.rentcollegepads.com/city/columbus-...,Null,NaN
1523,"50 W Oakland Ave, Columbus, OH 43201, United S...",W,4,1,1,599-599,0,NaN,https://www.rentcollegepads.com/city/columbus-...,Off-Street Parking,WasherDryer In-Unit
1524,"201 E 16th Ave Columbus, OH 43201 USA",E,5x2,5,2,2500-2500,2,NaN,https://www.rentcollegepads.com/city/columbus-...,Off-Street Parking,"WasherDryer In-Unit\n ,Washer Dryer Hook-Ups"


In [344]:
merged['Furnished'] = merged["Furnished"].apply(lambda x: "Yes" if x is not np.nan else x)
merged['Furnished'] = merged["Furnished"].apply(lambda x: "No" if x is np.nan else x)


,location_name,location,name,beds,baths,price,number,Furnished,url,Parking_final,Laundry_Wash,Furnished_stuff
0,"442 East Northwood Avenue Columbus, OH 43201 USA",East,Unfurnished,2,1,790-790,1,Yes,https://www.rentcollegepads.com/city/columbus-...,Off-Street Parking,"Laundry Room,On-Site Laundry",Yes
1,"442 East Northwood Avenue Columbus, OH 43201 USA",East,Furnished,2,1,850-850,1,Yes,https://www.rentcollegepads.com/city/columbus-...,Off-Street Parking,"Laundry Room,On-Site Laundry",Yes
2,"250 W Norwich Ave Columbus, OH 43201",W,Duo,2,2,1130-1140,1,Yes,https://www.rentcollegepads.com/city/columbus-...,Garage Parking,WasherDryer In-Unit,Yes
3,"250 W Norwich Ave Columbus, OH 43201",W,Quartet 1,4,4,950-965,1,Yes,https://www.rentcollegepads.com/city/columbus-...,Garage Parking,WasherDryer In-Unit,Yes
4,"250 W Norwich Ave Columbus, OH 43201",W,Quartet 2,4,4,920-955,1,Yes,https://www.rentcollegepads.com/city/columbus-...,Garage Parking,WasherDryer In-Unit,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...
1521,"155 W 10th Ave, Columbus, OH 43201, United States",W,None,6,2,0,1,No,https://www.rentcollegepads.com/city/columbus-...,On Street Parking Only,Laundry in Apartment,No
1522,"393 W 8th Ave Columbus, OH 43201",W,393 W 8th Ave,7,3,4000-4000,NaN,No,https://www.rentcollegepads.com/city/columbus-...,Null,NaN,No
1523,"50 W Oakland Ave, Columbus, OH 43201, United S...",W,4,1,1,599-599,0,No,https://www.rentcollegepads.com/city/columbus-...,Off-Street Parking,WasherDryer In-Unit,No
1524,"201 E 16th Ave Columbus, OH 43201 USA",E,5x2,5,2,2500-2500,2,No,https://www.rentcollegepads.com/city/columbus-...,Off-Street Parking,"WasherDryer In-Unit\n ,Washer Dryer Hook-Ups",No


In [384]:
merged[merged['Parking_final'] == 'Null']

,location_name,location,beds,baths,price,number,Furnished,url,Parking_final,Laundry_Wash
24,"15 E Lane Ave, Columbus, OH 43201, United States",E,2,2,0,NaN,No,https://www.rentcollegepads.com/city/columbus-...,Null,NaN
25,"15 E Lane Ave, Columbus, OH 43201, United States",E,2,2,0,NaN,No,https://www.rentcollegepads.com/city/columbus-...,Null,NaN
26,"15 E Lane Ave, Columbus, OH 43201, United States",E,3,3,0,NaN,No,https://www.rentcollegepads.com/city/columbus-...,Null,NaN
51,"62 E 13th Ave Columbus, OH 43201",E,4,1,2060-2060,NaN,No,https://www.rentcollegepads.com/city/columbus-...,Null,NaN
52,"62 E 13th Ave Columbus, OH 43201",E,4,1,2060-2060,NaN,No,https://www.rentcollegepads.com/city/columbus-...,Null,NaN
...,...,...,...,...,...,...,...,...,...,...
1497,"1382 Indianola Ave, Columbus, OH 43201, United...",Null,3,2,0,NaN,No,https://www.rentcollegepads.com/city/columbus-...,Null,NaN
1498,"1382 Indianola Ave, Columbus, OH 43201, United...",Null,4,2,0,NaN,No,https://www.rentcollegepads.com/city/columbus-...,Null,NaN
1515,"2159 Indianola Avenue Columbus, OH 43201",Null,3,2,1350-1450,NaN,No,https://www.rentcollegepads.com/city/columbus-...,Null,WasherDryer In-Unit
1516,"2159 Indianola Avenue Columbus, OH 43201",Null,3,1,1300-1450,NaN,No,https://www.rentcollegepads.com/city/columbus-...,Null,WasherDryer In-Unit


In [372]:
merged['Parking_final'] = merged["Parking_final"].apply(lambda x: ",".join(x) if isinstance(x, list) else x)

In [373]:
merged['Parking_final'].unique()

array(['Off-Street Parking', 'Garage Parking', 'Surface Lot',
       'Indoor Bike Storage', 'Null', 'Call for Availability',
       '404 King Parking', nan, 'On-Site Parking', 'Parking Available',
       'off-street parking', 'Call For Availability',
       'Off Street Parking', 'Free Off Street Parking', 'Street Parking',
       'Free off street parking', 'On Street Parking Only',
       'Assigned Parking Space', 'Off Street Parking Available',
       'off-street parking available', 'Outdoor Bike Storage',
       'Gated Parking', '2 Assigned Parking Spaces (1 Double Lane)',
       'Garage & Surface Available', 'Driveway Parking',
       'Assigned Off-Street Parking Spaces', 'Underground Parking'],
      dtype=object)

In [389]:
merged['Parking_final'] = merged["Parking_final"].apply(lambda x: "No" if x == "On Street Parking Only" else x)

In [395]:
list_parking = merged['Parking_final'].unique()
list_parking

array(['Off-Street Parking', 'Garage Parking', 'Surface Lot',
       'Indoor Bike Storage', 'Null', 'Call for Availability',
       '404 King Parking', nan, 'On-Site Parking', 'Parking Available',
       'off-street parking', 'Call For Availability',
       'Off Street Parking', 'Free Off Street Parking', 'Street Parking',
       'Free off street parking', 'No', 'Assigned Parking Space',
       'Off Street Parking Available', 'off-street parking available',
       'Outdoor Bike Storage', 'Gated Parking',
       '2 Assigned Parking Spaces (1 Double Lane)',
       'Garage & Surface Available', 'Driveway Parking',
       'Assigned Off-Street Parking Spaces', 'Underground Parking'],
      dtype=object)

In [401]:
list_parking = list_parking.tolist()

In [412]:
merged['Parking_final'] = merged["Parking_final"].apply(lambda x: "Yes" if x in list_parking else x)

In [413]:
merged

,location_name,location,beds,baths,price,number,Furnished,url,Parking_final,Laundry_Wash
0,"442 East Northwood Avenue Columbus, OH 43201 USA",East,2,1,790-790,1,Yes,https://www.rentcollegepads.com/city/columbus-...,Yes,"Laundry Room,On-Site Laundry"
1,"442 East Northwood Avenue Columbus, OH 43201 USA",East,2,1,850-850,1,Yes,https://www.rentcollegepads.com/city/columbus-...,Yes,"Laundry Room,On-Site Laundry"
2,"250 W Norwich Ave Columbus, OH 43201",W,2,2,1130-1140,1,Yes,https://www.rentcollegepads.com/city/columbus-...,Yes,WasherDryer In-Unit
3,"250 W Norwich Ave Columbus, OH 43201",W,4,4,950-965,1,Yes,https://www.rentcollegepads.com/city/columbus-...,Yes,WasherDryer In-Unit
4,"250 W Norwich Ave Columbus, OH 43201",W,4,4,920-955,1,Yes,https://www.rentcollegepads.com/city/columbus-...,Yes,WasherDryer In-Unit
...,...,...,...,...,...,...,...,...,...,...
1521,"155 W 10th Ave, Columbus, OH 43201, United States",W,6,2,0,1,No,https://www.rentcollegepads.com/city/columbus-...,No,Laundry in Apartment
1522,"393 W 8th Ave Columbus, OH 43201",W,7,3,4000-4000,NaN,No,https://www.rentcollegepads.com/city/columbus-...,Null,NaN
1523,"50 W Oakland Ave, Columbus, OH 43201, United S...",W,1,1,599-599,0,No,https://www.rentcollegepads.com/city/columbus-...,Yes,WasherDryer In-Unit
1524,"201 E 16th Ave Columbus, OH 43201 USA",E,5,2,2500-2500,2,No,https://www.rentcollegepads.com/city/columbus-...,Yes,"WasherDryer In-Unit\n ,Washer Dryer Hook-Ups"


In [460]:
merged['number'].fillna(0, inplace=True)
merged['number'] = merged['number'].astype(int)
merged['Parking_final'] = np.where((merged['Parking_final'] == 'Yes') & (merged['number'] > 0),
                     "Yes", "No")

In [462]:
merged = merged.drop(['number','url','location_name'],axis = 1)
merged

,location,beds,baths,price,Furnished,Parking_final,Laundry_Wash
0,East,2,1,790-790,Yes,Yes,"Laundry Room,On-Site Laundry"
1,East,2,1,850-850,Yes,Yes,"Laundry Room,On-Site Laundry"
2,W,2,2,1130-1140,Yes,Yes,WasherDryer In-Unit
3,W,4,4,950-965,Yes,Yes,WasherDryer In-Unit
4,W,4,4,920-955,Yes,Yes,WasherDryer In-Unit
...,...,...,...,...,...,...,...
1521,W,6,2,0,No,No,Laundry in Apartment
1522,W,7,3,4000-4000,No,No,NaN
1523,W,1,1,599-599,No,No,WasherDryer In-Unit
1524,E,5,2,2500-2500,No,Yes,"WasherDryer In-Unit\n ,Washer Dryer Hook-Ups"


In [463]:
merged['Laundry_Wash'].unique()

array(['Laundry Room,On-Site Laundry', 'WasherDryer In-Unit', nan,
       'On-Site Laundry',
       'Laundry Room,New Laundry Machines, On-Site Laundry',
       'Laundry in building', 'WasherDryer In-Unit,Washer Dryer Hook-Ups',
       'Laundry In Unit', 'On-Site Laundry,Laundry Room',
       'Washer Dryer Hook-Ups', 'Laundry in Apartment',
       'Laundry in House', 'Laundry in Building', ' On-Site Laundry',
       'Laundry in Unit', 'WasherDryer In-Unit\n ', ' Laundry In Unit',
       'WasherDryer In-Unit\n ,Washer Dryer Hook-Ups', 'Laundry on site'],
      dtype=object)

In [488]:
list_laundry = ['WasherDryer In-Unit','Laundry in Apartment','Laundry In Unit',
                'WasherDryer In-Unit\n','Washer Dryer Hook-Ups','Laundry in House',
                'WasherDryer In-Unit,Washer Dryer Hook-Ups','WasherDryer In-Unit\n ,Washer Dryer Hook-Ups',
               'Laundry in Unit',' Laundry In Unit','WasherDryer In-Unit\n ']

In [489]:
list_laundry

['WasherDryer In-Unit',
 'Laundry in Apartment',
 'Laundry In Unit',
 'WasherDryer In-Unit\n',
 'Washer Dryer Hook-Ups',
 'Laundry in House',
 'WasherDryer In-Unit,Washer Dryer Hook-Ups',
 'WasherDryer In-Unit\n ,Washer Dryer Hook-Ups',
 'Laundry in Unit',
 ' Laundry In Unit',
 'WasherDryer In-Unit\n ']

In [490]:
merged["Laundry_Wash"] = merged["Laundry_Wash"].apply(lambda x: "unit" if x in list_laundry else x)

In [491]:
merged

,location,beds,baths,price,Furnished,Parking_final,Laundry_Wash
0,East,2,1,790-790,Yes,Yes,building
1,East,2,1,850-850,Yes,Yes,building
2,W,2,2,1130-1140,Yes,Yes,unit
3,W,4,4,950-965,Yes,Yes,unit
4,W,4,4,920-955,Yes,Yes,unit
...,...,...,...,...,...,...,...
1521,W,6,2,0,No,No,unit
1522,W,7,3,4000-4000,No,No,NaN
1523,W,1,1,599-599,No,No,unit
1524,E,5,2,2500-2500,No,Yes,unit


In [492]:
list_laundry_building = ['Laundry Room,On-Site Laundry','On-Site Laundry','Laundry Room,New Laundry Machines, On-Site Laundry',
                        'Laundry in building','On-Site Laundry,Laundry Room','Laundry in Building',
                        'On-Site Laundry','Laundry on site',' On-Site Laundry'
                        'WasherDryer In-Unit\n ',' On-Site Laundry']

In [493]:
merged["Laundry_Wash"] = merged["Laundry_Wash"].apply(lambda x: "building" if x in list_laundry_building else x)
merged

,location,beds,baths,price,Furnished,Parking_final,Laundry_Wash
0,East,2,1,790-790,Yes,Yes,building
1,East,2,1,850-850,Yes,Yes,building
2,W,2,2,1130-1140,Yes,Yes,unit
3,W,4,4,950-965,Yes,Yes,unit
4,W,4,4,920-955,Yes,Yes,unit
...,...,...,...,...,...,...,...
1521,W,6,2,0,No,No,unit
1522,W,7,3,4000-4000,No,No,NaN
1523,W,1,1,599-599,No,No,unit
1524,E,5,2,2500-2500,No,Yes,unit


In [494]:
merged['Laundry_Wash'].unique()

array(['building', 'unit', nan], dtype=object)

In [504]:
merged['price'] = merged["price"].apply(lambda x: x.split('-')[0])
merged

,location,beds,baths,price,Furnished,Parking_final,Laundry_Wash
0,East,2,1,790,Yes,Yes,building
1,East,2,1,850,Yes,Yes,building
2,W,2,2,1130,Yes,Yes,unit
3,W,4,4,950,Yes,Yes,unit
4,W,4,4,920,Yes,Yes,unit
...,...,...,...,...,...,...,...
1521,W,6,2,0,No,No,unit
1522,W,7,3,4000,No,No,NaN
1523,W,1,1,599,No,No,unit
1524,E,5,2,2500,No,Yes,unit


In [515]:
merged['location'] = merged['location'].apply(lambda x: "West" if x == "W" else x)
merged['location']  = merged['location'].apply(lambda x: "East" if x == "E" else x)
merged['location']  = merged['location'].apply(lambda x: "North" if x == "N" else x)
merged['location']  = merged['location'].apply(lambda x: "South" if x == "S" else x)

In [516]:
merged['location'].unique()

array(['East', 'West', 'Null', 'North', 'South'], dtype=object)

In [521]:
merged.to_csv('merged.csv')

In [518]:
merged_raw = pd.merge(df,additional,on = "location_name",how = "left")

In [522]:
merged_raw.to_csv('merged_raw.csv')